# 2. Load Data

In [1]:
import pandas as pd; from sqlalchemy import create_engine
engine = create_engine("mssql+pyodbc://localhost/DW_PROJECT?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes")

query = """SELECT f.JobPosting_Key,f.job_no_degree_mention,f.job_health_insurance,f.job_work_from_home,CAST(t.job_title_short AS VARCHAR(300)) AS job_title_short,CAST(c.company_name AS VARCHAR(300)) AS company_name,CAST(l.job_location AS VARCHAR(300)) AS job_location,CAST(co.CountryName AS VARCHAR(200)) AS CountryName,CAST(s.job_schedule_type AS VARCHAR(200)) AS job_schedule_type,CAST(v.job_via AS VARCHAR(200)) AS job_via,d.FullDate AS Job_Posted_Date,CAST(sk.Skill_Name AS VARCHAR(300)) AS Skill_Name,CAST(st.SkillType_Name AS VARCHAR(200)) AS SkillType_Name FROM Fact_Job_Postings f LEFT JOIN Dim_Title t ON f.Title_Key=t.Title_Key LEFT JOIN Dim_Company c ON f.Company_Key=c.Company_Key LEFT JOIN Dim_Location l ON f.Location_Key=l.Location_Key LEFT JOIN Dim_Country co ON l.Country_Key=co.Country_Key LEFT JOIN Dim_ScheduleType s ON f.ScheduleType_Key=s.ScheduleType_Key LEFT JOIN Dim_Via v ON f.Via_Key=v.Via_Key LEFT JOIN DimDate d ON f.Date_ID=d.Date_ID LEFT JOIN Bridge_JobPosting_Skill b ON f.JobPosting_Key=b.JobPosting_Key LEFT JOIN Dim_Skill sk ON b.Skill_Key=sk.Skill_Key LEFT JOIN Dim_SkillType st ON sk.SkillType_Key=st.SkillType_Key ORDER BY f.JobPosting_Key;"""

df = pd.read_sql(query, engine)
df.head()

,JobPosting_Key,job_no_degree_mention,job_health_insurance,job_work_from_home,job_title_short,company_name,job_location,CountryName,job_schedule_type,job_via,Job_Posted_Date,Skill_Name,SkillType_Name
0,1,False,False,False,senior data engineer,boehringer ingelheim,watertown,united states,full-time,work nearby,2023-06-16,None,None
1,2,False,False,False,data analyst,hewlett packard enterprise,"guadalajara, jalisco",mexico,full-time,bebee méxico,2023-01-14,r,programming
2,2,False,False,False,data analyst,hewlett packard enterprise,"guadalajara, jalisco",mexico,full-time,bebee méxico,2023-01-14,nosql,programming
3,2,False,False,False,data analyst,hewlett packard enterprise,"guadalajara, jalisco",mexico,full-time,bebee méxico,2023-01-14,sql,programming
4,2,False,False,False,data analyst,hewlett packard enterprise,"guadalajara, jalisco",mexico,full-time,bebee méxico,2023-01-14,python,programming


# 3. Exploratory Data Analysis

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3404826 entries, 0 to 3404825
Data columns (total 13 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   JobPosting_Key         int64 
 1   job_no_degree_mention  bool  
 2   job_health_insurance   bool  
 3   job_work_from_home     bool  
 4   job_title_short        object
 5   company_name           object
 6   job_location           object
 7   CountryName            object
 8   job_schedule_type      object
 9   job_via                object
 10  Job_Posted_Date        object
 11  Skill_Name             object
 12  SkillType_Name         object
dtypes: bool(3), int64(1), object(9)
memory usage: 269.5+ MB


In [4]:
df.isnull().sum()

JobPosting_Key                0
job_no_degree_mention         0
job_health_insurance          0
job_work_from_home            0
job_title_short               0
company_name                  0
job_location                  0
CountryName                   0
job_schedule_type             0
job_via                       0
Job_Posted_Date               0
Skill_Name               195646
SkillType_Name           195646
dtype: int64

In [5]:
df.describe()

,JobPosting_Key
count,3.404826e+06
mean,3.737073e+05
std,2.236988e+05
min,1.000000e+00
25%,1.767900e+05
50%,3.703610e+05
75%,5.663610e+05
max,7.719030e+05


In [6]:
df.select_dtypes(include='object').nunique()

job_title_short          10
company_name         127369
job_location          14777
CountryName             160
job_schedule_type        28
job_via                7669
Job_Posted_Date         365
Skill_Name              240
SkillType_Name           10
dtype: int64

In [7]:
df.duplicated().sum()

0

In [8]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Job Postings", minimal=True)
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:00<00:00, 17.04it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

# 4. Data Preparation

In [ ]:
# Convert Job_Posted_Date into a datetime object
df['Job_Posted_Date'] = pd.to_datetime(df['Job_Posted_Date'])

# Extract the month (year-month format) from the posting date
df['month'] = df['Job_Posted_Date'].dt.to_period('M')


# -----------------------------------------------
# Group skills by JobPosting_Key
# Each job posting may appear multiple times with different skills
# This groups all skills of the same job into a list
skills_df = df.groupby('JobPosting_Key')['Skill_Name'].apply(list).reset_index()


# Select one row per job posting (job title, country, company)
# drop_duplicates ensures we don't have repeated entries
jobs = df[['JobPosting_Key', 'job_title_short', 'CountryName', 'company_name']].drop_duplicates()


# Merge job info with their list of skills
final = jobs.merge(skills_df, on='JobPosting_Key')


# Convert the list of skills into one single string (for NLP/text processing)
final['skill_text'] = final['Skill_Name'].apply(
    lambda x: ' '.join([s for s in x if isinstance(s, str)]) if isinstance(x, list) else ''
)


# Count how many times each skill appears in the dataset
skill_counts = df['Skill_Name'].value_counts()


# Count the number of job postings per skill per month
skill_trend = df.groupby(['Skill_Name', 'month']).JobPosting_Key.nunique()


# ------------------------------------------------
# TF-IDF Vectorization
# Convert the skill_text column into numerical vectors
# TF-IDF measures how important each skill word is across all job postings
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

# Apply TF-IDF and convert the result into a DataFrame
df_tfidf = pd.DataFrame(
    tfidf.fit_transform(final['skill_text']).toarray(),
    columns=tfidf.get_feature_names_out()
)



In [ ]:
# Convert job_work_from_home to a clean binary target (0 = onsite, 1 = remote)
# Keep only one row per JobPosting_Key with the remote flag information
final_target = df[['JobPosting_Key', 'job_work_from_home']].drop_duplicates()

# Convert job_work_from_home into a binary flag
# Some datasets use -1, 0, 1 → here we convert ONLY 1 to remote
final_target['remote_flag'] = final_target['job_work_from_home'].apply(
    lambda x: 1 if x == 1 else 0
)

# Group skills for each job posting

# Each job may appear multiple times with different Skill_Name values
# This groups all skills for the same job into a Python list
skills_df = df.groupby('JobPosting_Key')['Skill_Name'].apply(list).reset_index()


# Extract job info (title, country, company)

# Each job posting may have duplicates due to multiple skills
# We keep only unique rows
jobs = df[['JobPosting_Key', 'job_title_short', 'CountryName', 'company_name']] \
       .drop_duplicates()

#Merge job information with the list of skills
final = jobs.merge(skills_df, on='JobPosting_Key')


# Convert list of skills into a single text string

# Example: ["Python", "SQL"] → "Python SQL"
# This is needed for TF-IDF text vectorization
final['skill_text'] = final['Skill_Name'].apply(
    lambda x: ' '.join([s for s in x if isinstance(s, str)]) if isinstance(x, list) else ''
)


# Add the remote / onsite target to the final dataset
final = final.merge(final_target[['JobPosting_Key', 'remote_flag']],
                    on='JobPosting_Key')

# Remove jobs that have no skills (empty skill_text)
final = final[final['skill_text'].str.strip() != '']

# Count how many jobs are remote vs onsite
job_counts = final['remote_flag'].value_counts()

# Print the result
print(job_counts)


remote_flag
0    522537
1     53720
Name: count, dtype: int64


In [11]:
final.to_csv("prepared_jobs_dataset.csv", index=False)
print("CSV file exported successfully!")



CSV file exported successfully!
